# Ré-entrainement du modèle

Ce Notebook permet de ré-entrainer le modèle en utilisant une base de données de mails avec des annotations corrigées par les exploitants

In [1]:
import warnings
warnings.filterwarnings("ignore")

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import exchangelib
from exchangelib import DELEGATE, Account, Credentials, Configuration

import datetime
from datetime import datetime
from datetime import date

In [ ]:
adresse_mail = ''
password = ''
fichier_model = ''
fichier_features = ''
model = joblib.load(fichier_model)
features = np.load(fichier_features)

In [12]:
## Les cordonnées de la boite mails

creds = Credentials(
    username="massermouh@groupe-idea.com", 
    password="0idcili1A*"
)

config = Configuration(server='outlook.office.com', credentials=creds)

account = Account(
    primary_smtp_address="massermouh@groupe-idea.com",
    autodiscover=False, 
    config=config,
    access_type=DELEGATE
)

## Vous voulez extraire les mails reçus depuis quelle dates?

date_début = date(2022, 7, 2)
now = datetime.now().date()
delta = now - date_début
h = delta.days*24

In [3]:
def html_to_text(html):
    
    soup = BeautifulSoup(html, features="html.parser")

    for script in soup(["script", "style"]):
        script.extract()  
    text = soup.get_text()

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    
    return text

In [13]:
## Indice du dossier d'importance 

dossiers = []
idxs = []
for f in account.inbox.walk():
    dossiers.append(f.name)
    idxs.append(f.id)
    
j = dossiers.index('TO DO')
folder_id = idxs[j]

In [14]:
folder_id

'AAMkADc4ODRlZTUxLWNkOGUtNDI4ZC05NDVkLTFjNDM0M2EwY2I4YwAuAAAAAAC8pQwzX7znQav762WrbYGwAQBQBcoV6pfRTYxoxbOYpm2AAAARPZEiAAA='

In [18]:
# sub_sub_folder = account.inbox / 'TO DO'
# for item in sub_sub_folder.all().order_by('-datetime_received')[:1]:
#     print(item.body)

In [20]:
from datetime import timedelta
from exchangelib import UTC_NOW

mails = pd.DataFrame(columns=['Subject','Date_Sent','Body','From (address)','To (address)','Class'])
                      
                      
since = UTC_NOW() - timedelta(hours=h)
for item in account.inbox.all()\
        .filter(datetime_received__gt=since)\
        .order_by('-datetime_received'):
    l=[]
    l.append(item.subject)
    l.append(item.datetime_received)
    b = html_to_text(item.unique_body)
    b = b.replace('\n',' ')
    l.append(b)
    l.append(item.sender.email_address)
    l.append(item.display_to)
    l.append(0)
    mails.loc[len(mails)] = l

sub_sub_folder = account.inbox / 'TO DO'
for item in sub_sub_folder.all()\
        .filter(datetime_received__gt=since)\
        .order_by('-datetime_received'):

    l=[]
    l.append(item.subject)
    l.append(item.datetime_received)
    b = html_to_text(item.unique_body)
    b = b.replace('\n',' ')
    l.append(b)
    l.append(item.sender.email_address)
    l.append(item.display_to)
    l.append(1)
    
    
    
    
    mails.loc[len(mails)] = l
    
mails['Date_Sent'] = pd.to_datetime(mails.Date_Sent).dt.tz_localize(None)
mails['Date_Sent'] = pd.to_datetime(mails['Date_Sent'])
mails.sort_values(by='Date_Sent')

,Subject,Date_Sent,Body,From (address),To (address),Class
40,Votre e-mail de synthèse,2022-07-04 00:35:14,"Privé pour vous Bonjour Assermouh Meryam, Déco...",viva-noreply@microsoft.com,Assermouh Meryam,0
39,Récapitulatif quotidien de vos tâches,2022-07-04 04:00:50,"Bonjour Assermouh Meryam, Nous espérons que vo...",viva-noreply@microsoft.com,Assermouh Meryam,0
38,L'hebdo myIDEA !,2022-07-04 07:13:19,L'info qu'il ne fallait pas manquer IDEA lance...,DLancelot@groupe-idea.com,Liste Groupe - IDEA,0
37,RE: Test technique IDEA,2022-07-04 10:25:03,"Bonjour, Comme convenu, Je vous joints mon tra...",KElAyech@groupe-idea.com,Assermouh Meryam,0
36,Annulé: IA by IDEA : Masterplan,2022-07-04 13:36:04,"Bonjour à tous, Je vous propose d'avoir un cré...",ADUFOUDEKERDANIEL@groupe-idea.com,Delacour Vincent; Bernicot Jean-Baptiste; Card...,0
35,Annulé: IA by IDEA : Masterplan,2022-07-04 13:36:04,"Bonjour à tous, Je vous propose d'avoir un cré...",ADUFOUDEKERDANIEL@groupe-idea.com,Delacour Vincent; Bernicot Jean-Baptiste; Card...,0
34,Annulé: IA by IDEA : Masterplan,2022-07-04 13:36:05,"Bonjour à tous, Je vous propose d'avoir un cré...",ADUFOUDEKERDANIEL@groupe-idea.com,Delacour Vincent; Bernicot Jean-Baptiste; Card...,0
33,Récapitulatif quotidien de vos tâches,2022-07-05 03:24:03,"Bonjour Assermouh Meryam, Faites la différence...",viva-noreply@microsoft.com,Assermouh Meryam,0
32,Daily - Team IA,2022-07-05 06:34:47,Bonjour à tous Désolée pour l’heure matinale p...,MBruggeman@groupe-idea.com,El Ayech Khaled; Assermouh Meryam,0
31,Rapport utilisateur : 1 graymail(s) et 0 indés...,2022-07-06 04:47:04,[Email émis par un expéditeur externe au group...,user.account@proginov.eu,Assermouh Meryam,0


## Preparing the data

In [27]:
matrice = pd.DataFrame()
matrice["text"] = mails['From (address)'].astype(str) + mails["Subject"].astype(str) + mails["Body"].astype(str)
messages = matrice["text"]

keys = ['longeur','nombre_de_destinataires','mots_maj','lettres_maj','punct_counts','longeur_moy_mots','ratios','uniques','nb_digits','nb_ref']+features

Z = []

for i, row in mails.iterrows():
    
    d = dict.fromkeys(keys)
    b = row['Body']
    text = str(row['Subject']) + str(row['From (address)']) + b
    ## Preprocessing : 

    #Preprocess text messages

    # Replace URLs with 'webaddress'
    processed = text.str.replace(r'^http\://[a-zA-Z0-9\-\.]+\.[a-zA-Z]{2,3}(/\S*)?$',
                                      'webaddress')

    # Replace money symbols with 'moneysymb' (£ can by typed with ALT key + 156)
    processed = processed.str.replace(r'£|\$', 'moneysymb')

    # Replace 10 digit phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
    processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                      'phonenumbr')

    # Replace numbers with 'numbr'
    processed = processed.str.replace(r'\d+(\.\d+)?', 'numbr')

    # Remove punctuation
    processed = processed.str.replace(r'[^\w\d\s]', ' ')

    # Replace whitespace between terms with a single space
    processed = processed.str.replace(r'\s+', ' ')

    # Remove leading and trailing whitespace
    processed = processed.str.replace(r'^\s+|\s+?$', '')

    # change words to lower case 
    processed = processed.str.lower()

    # remove french stop words from text messages

    stop_words = set(stopwords.words('french'))

    processed = processed.apply(lambda x: ' '.join(
        term for term in x.split() if term not in stop_words))

    # remove english stop words from text messages

    stop_words = set(stopwords.words('english'))
    # stop_words.add('bonjour')
    stop_words.add('please')
    stop_words.add('mailto')
    stop_words.add('mail')
    stop_words.add('numbr')
    # stop_words.add('cordialement')
    stop_words.add('e')
    stop_words.add('virus')
    stop_words.add('antiphishing')
    stop_words.add('merci')
    stop_words.add('com')
    stop_words.add('fr')
    stop_words.add('https')
    stop_words.add('bjenumbrywvybnumbrbhcmnaznumbrjvdxbllwlkzweuynumbrt')

    processed = processed.apply(lambda x: ' '.join(
        term for term in x.split() if term not in stop_words))
    
    for e in keys[10:]:
        if e in text.split():
            d[e] = True
        else:
            d[e] = False
    
    # 1 - longeur du mail

    d["longeur"] = len(b)

    # 2 - Nombre de destinataires/copies

    nb_destinataires = []
    w = str(item.sender.email_address).lower()
#         w = replace_substring(w, s, '@IDEA')  

    d['nombre_de_destinataires'] = w.count('@')

    # 3 - nombre de mots en majuscule

    maj = sum(map(str.isupper, b.split())) + sum(map(str.isupper, str(item.subject).split()))
    d['mots_maj'] = maj

    #4 - nombre de lettres majuscules

    maj = sum(i.isupper() for i in b)
    d['lettres_maj'] = maj

    # 5 -  nombre de ponctuations
    punct_counts = []

    import string 

    punct = string.punctuation

    punct_count = 0
    for element in punct :
        punct_count = punct_count + b.count(element)
        
    d['punct_counts'] = punct_count

    # # 7 -  longeur moyenne du mot

    # from statistics import mean
    
    ls = [len(x) for x in b.split()]
    lg = np.mean(ls)

    d['longueur_moy_mots'] = lg

    # # 9 -  ratio stop words

    x = sum([b.split().count(x) for x in (set(stopwords.words('french')) | set(stopwords.words('english')))])/len(str(row['Body']).split())

    rat = x/len(b.split())
    ratios.append(rat)

    b['ratios'] = ratios

    # # 10 - nombre de mots uniques

    unique = len(set(str(row['Body']).split()))

    d['uniques'] = unique
    
    # 11 -  nombre de digits

    nb_digits = []
    digits = list(range(10))
    m = 0
    for e in digits :
        m = m + b.count(str(m))

    d['nb_digits'] = m    

    # 12 - nombre de mots contenants digits et lettres

    import string

    x = 0
    for element in b.split() :
        if len(set(list(element))&set(list(string.ascii_lowercase)))*len(set(list(range(10)))&set(list(element))) !=0 :
            x = x + 1
    d['nb_ref'] = x
    
    Z.append((d,row['Class']))


NameError: name 'features' is not defined

In [ ]:
#Splitting the data

all_indices = list(range(len(Z)))

train_indices, test_indices = model_selection.train_test_split(all_indices, test_size = 0.25, random_state = 42)

#print(train_indices)
training = [Z[i] for i in train_indices]
testing = [Z[i] for i in test_indices]


In [ ]:
# train the model on the training data
model.train(training)

# and test on the testing dataset!
accuracy = nltk.classify.accuracy(model, testing)*100
print("Model Accuracy: {}".format(accuracy))

## Saving the model

In [ ]:
model_file = './Documents/Priorisation de mails/model_magasinX.joblib'
joblib.dump(model, model_file)